In [1]:
import matplotlib.pyplot as plt
from skopt.learning import ExtraTreesRegressor as opt_ETR
import random
import numpy as np
random.seed(1126)
np.random.seed(1126)

from scipy.stats import norm
from script.machine_learning_model import *

In [2]:
def rand_search(ninit, x, y, random_state = 929):
    random.seed(random_state)
    ninit = ninit # number of first point
    niter = len(x) # number of iteration
    true_max = np.max(y)
    order = list(range(len(x)))
    random.shuffle(order)

    y_list = []
    z_list = []
    for i in range(ninit):
        ridx = order[i]
        y_list.append(y.iloc[ridx])
        
    cur_max = np.array(y_list).max()
    for j in range(400):
        ridx = order[j + ninit]
        y_list.append(y.iloc[ridx])
        yp = np.array(y_list)
        cur_max = np.max(yp)
        z_list.append(cur_max)
        
        if cur_max >= true_max:
            print('max found', j)
            
        print('iter:{0}, current_max:{1}'.format(j,cur_max))
     
    return z_list

def posterior(x, p_x, p_y, model):
    if len(p_x.shape) == 1:
        model.fit(p_x.reshape(-1, 1), p_y)
        mu, sigma = model.predict(x.reshape(-1,1), return_std=True)
    else:
        model.fit(p_x, p_y)
        mu, sigma = model.predict(x, return_std=True)
    ind = np.where(sigma == 0.)
    sigma[ind] = 1e-5
    return mu, sigma

def EI(mu, sigma, cur_max):
    Z = (mu - cur_max) / sigma
    ei = (mu - cur_max) * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

def bo(ninit, model, x, y, random_state = 929):
    random.seed(random_state)
    ninit = ninit # number of first point
    niter = len(x) # number of iteration
    true_max = np.max(y)
    order = list(range(len(x)))
    random.shuffle(order)

    x_list = []
    y_list = []
    z_list = []
    used = set()
    
    for i in range(ninit):
        ridx = order[i]
        x_list.append(x.iloc[ridx, :])
        y_list.append(y.iloc[ridx])
        used.add(ridx)

    for j in range(400):
        xp = np.array(x_list)
        yp = np.array(y_list)
        cur_max = np.max(yp)
        # fit surrogate model
        model.fit(xp, yp)
        _mu, sigma = model.predict(x, return_std=True)
        mu = _mu.reshape(-1)
        ind = np.where(sigma == 0.)
        sigma[ind] = 1e-5
        # compute EI
        Z = (mu - cur_max) / sigma
        ei = (mu - cur_max) * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] == 0.0
        idlist = np.argsort(ei)[::-1]
        p = 0
        max_idx = idlist[p]
        while max_idx in used:
            p += 1
            max_idx = idlist[p]
        
        used.add(max_idx)
        x_list.append(x.iloc[max_idx, :])
        y_list.append(y.iloc[max_idx])
        z_list.append(cur_max)
            
        print('iter:{0}, current_max:{1}'.format(j,cur_max))
        
    return x_list, y_list, z_list

In [ ]:
feat, target = data_load("conv")
rand = []
for i in range(10):
    rand.append(rand_search(10, feat, target, random_state = 1126+7*i))
    
model = opt_ETR(n_estimators=500, n_jobs=-1,random_state=1126)
res_model_bo_et_conv = []
for i in range(10):
    res_model_bo_et_conv.append(bo(10,model,feat, target,random_state=i*10))
    

feat, target = data_load("prop")
model = opt_ETR(n_estimators=500, n_jobs=-1,random_state=1126)
res_model_bo_et = []
for i in range(10):
    res_model_bo_et.append(bo(10,model,feat, target,random_state=i*10))
 

feat, target = data_load("prop2")
model = opt_ETR(n_estimators=500, n_jobs=-1,random_state=929)
res_model_bo_et_prop2 = []
for i in range(10):
    res_model_bo_et_prop2.append(bo(10,model,feat, target,random_state=i*10))

    
import sklearn.gaussian_process as gp
kernel = gp.kernels.Matern(nu = 2.5)
model = gp.GaussianProcessRegressor(kernel=kernel,
                                    alpha=1e-2,
                                    n_restarts_optimizer=10,
                                    normalize_y=True,
                                    random_state=1126)
res_model_bo_gp = []
for i in range(10):
    res_model_bo_gp.append(bo(10,model,feat,target,random_state=i*10))
    
rand_pred = np.array(rand).mean(axis=0)
et_mean_pred= np.array([x[2] for x in res_model_bo_et]).mean(axis=0)
et_mean_pred_conv= np.array([x[2] for x in res_model_bo_et_conv]).mean(axis=0)
gp_mean_pred= np.array([x[2] for x in res_model_bo_gp]).mean(axis=0)
et_mean_pred_prop2= np.array([x[2] for x in res_model_bo_et_prop2]).mean(axis=0)


plt.figure(figsize=(6,3), dpi=1200)
plt.plot(rand_pred, label='Random')
plt.plot(et_mean_pred, label='SMAC (Conventional)')
plt.plot(et_mean_pred_conv, label='SMAC (Proposed)')
plt.plot(et_mean_pred_prop2, label='SMAC (Proposed2)')
plt.plot(gp_mean_pred, label = "BO(GBR)")
plt.legend()
plt.savefig('out/Figure_10.png', dpi = 1200, bbox_inches = 'tight')

iter:0, current_max:21.9
iter:1, current_max:21.9
iter:2, current_max:21.9
iter:3, current_max:21.9
iter:4, current_max:21.9
iter:5, current_max:21.9
iter:6, current_max:21.9
iter:7, current_max:21.9
iter:8, current_max:21.9
iter:9, current_max:21.9
iter:10, current_max:21.9
iter:11, current_max:21.9
iter:12, current_max:21.9
iter:13, current_max:21.9
iter:14, current_max:21.9
iter:15, current_max:21.9
iter:16, current_max:21.9
iter:17, current_max:21.9
iter:18, current_max:21.9
iter:19, current_max:21.9
iter:20, current_max:21.9
iter:21, current_max:21.9
iter:22, current_max:21.9
iter:23, current_max:21.9
iter:24, current_max:21.9
iter:25, current_max:21.9
iter:26, current_max:21.9
iter:27, current_max:21.9
iter:28, current_max:21.9
iter:29, current_max:21.9
iter:30, current_max:21.9
iter:31, current_max:21.9
iter:32, current_max:21.9
iter:33, current_max:21.9
iter:34, current_max:21.9
iter:35, current_max:21.9
iter:36, current_max:21.9
iter:37, current_max:21.9
iter:38, current_max:2